In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r "/content/drive/My Drive/research_lo_content_taxonomy_classification/model_euclidean_dual_bert_1_khan_acad/" /content/
!cp -r "/content/drive/My Drive/research_lo_content_taxonomy_classification/model_euclidean_dual_bert_2_khan_acad/" /content/

In [4]:
!pip install transformers==2.8.0

In [5]:
!pip install git+https://github.com/geoopt/geoopt.git
! pip install git+https://github.com/ferrine/hyrnn.git

  Cloning https://github.com/geoopt/geoopt.git to /tmp/pip-req-build-l32qvs7d
  Running command git clone -q https://github.com/geoopt/geoopt.git /tmp/pip-req-build-l32qvs7d
  Created wheel for geoopt: filename=geoopt-0.3.1-cp37-none-any.whl size=76168 sha256=d95c6bfd269b822163a3dba557760d8fa601224325d03387393d4a4ad2e0b19b
  Stored in directory: /tmp/pip-ephem-wheel-cache-up9zr1wh/wheels/10/df/30/e0d857f034c142ca5f38af048b62aae3da773b272553e5dd21
Successfully built geoopt
  Cloning https://github.com/ferrine/hyrnn.git to /tmp/pip-req-build-aaohawc8
  Running command git clone -q https://github.com/ferrine/hyrnn.git /tmp/pip-req-build-aaohawc8
  Created wheel for hyrnn: filename=hyrnn-0.0.0-cp37-none-any.whl size=13955 sha256=a719b1d85a11925de3d31213e901886f727c4018a9589a12fc671d25a83a0e7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_wfqck3y/wheels/24/c3/64/cc0e9d25d466081dc154a2a8843157f54d845b916b4ba66418
Successfully built hyrnn


In [6]:
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/train_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/test_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/val_khan_acad.csv" /content


In [7]:
import pandas as pd
train_data = pd.read_csv("train_khan_acad.csv")
val_data = pd.read_csv("val_khan_acad.csv")
test_data = pd.read_csv("test_khan_acad.csv")

train_data


,video_transcripts,hierarchy
0,In the last couple of videos we saw that we c...,math>>multivariable-calculus>>multivariable-de...
1,- What we're going to do in this video is gi...,science>>ap-biology>>natural-selection
2,"So once again, we have three equal, or we say...",math>>pre-algebra>>pre-algebra-equations-expre...
3,- Liz's math test included a survey question...,math>>engageny-alg-1>>alg1-2
4,- The following two equations form a linear s...,math>>algebra-home>>alg-system-of-equations
...,...,...
4183,- Hello everyone. So this is what I might ca...,math>>multivariable-calculus>>multivariable-de...
4184,- Let's try now to subtract some two-digit n...,math>>early-math>>cc-early-math-add-sub-100
4185,- Let's say that I have a circle. My best att...,math>>engageny-geo>>geo-5
4186,- So let's look at the female reproductive cy...,science>>health-and-medicine>>human-anatomy-an...


In [8]:
set(train_data["video_transcripts"].values).intersection(set(test_data["video_transcripts"].values))

set()

In [9]:
from google.colab import files

In [10]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [11]:
# final_data_1 = final_data.loc[0:71003,:]
# final_data_1

In [12]:
import transformers
print(transformers.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

2.8.0


In [13]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [14]:
train_data["hierarchy"].value_counts()

science>>health-and-medicine>>circulatory-system-diseases                   99
science>>health-and-medicine>>human-anatomy-and-physiology                  65
science>>health-and-medicine>>respiratory-system-diseases                   55
science>>health-and-medicine>>circulatory-system                            54
science>>health-and-medicine>>infectious-diseases                           52
                                                                            ..
math>>precalculus>>x9e81a4f98389efdf:vectors                                 1
math>>statistics-probability>>advanced-regression-inference-transforming     1
math>>old-ap-calculus-ab>>ab-derivatives-advanced                            1
math>>4th-grade-foundations-engageny>>4th-m5-engage-ny-foundations           1
math>>algebra-basics>>alg-basics-expressions-with-exponents                  1
Name: hierarchy, Length: 569, dtype: int64

In [15]:
test_data["hierarchy"].value_counts()

science>>health-and-medicine>>human-anatomy-and-physiology                                    24
science>>health-and-medicine>>circulatory-system-diseases                                     22
science>>health-and-medicine>>circulatory-system                                              17
science>>health-and-medicine>>infectious-diseases                                             11
science>>biology>>crash-course-bio-ecology                                                    11
                                                                                              ..
math>>cc-seventh-grade-math>>cc-7th-fractions-decimals                                         1
math>>cc-sixth-grade-math>>cc-6th-equations-and-inequalities                                   1
science>>high-school-biology>>hs-evolution                                                     1
math>>in-in-class-5th-math-cbse>>x91a8f6d2871c8046:multi-digit-multiplication-and-division     1
math>>algebra-basics>>alg-basi

In [16]:

# from sklearn.preprocessing import LabelEncoder

# LE = LabelEncoder()
# final_data['label'] = LE.fit_transform(final_data['board_syllabus'])
# final_data.head()

In [17]:
# def get_labels(prediction):
#     predicted_label =  LE.inverse_transform([prediction])
#     return predicted_label[0]

In [18]:
# get_labels(330)

In [19]:
# train_data = pd.concat([train_data,val_data])
# train_data

In [20]:
train_features = train_data["video_transcripts"]
test_features = test_data["video_transcripts"]
train_labels = train_data["hierarchy"]
test_labels = test_data["hierarchy"]
val_features = val_data["video_transcripts"]
val_labels = val_data["hierarchy"]

In [21]:
train_labels.value_counts()

science>>health-and-medicine>>circulatory-system-diseases                   99
science>>health-and-medicine>>human-anatomy-and-physiology                  65
science>>health-and-medicine>>respiratory-system-diseases                   55
science>>health-and-medicine>>circulatory-system                            54
science>>health-and-medicine>>infectious-diseases                           52
                                                                            ..
math>>precalculus>>x9e81a4f98389efdf:vectors                                 1
math>>statistics-probability>>advanced-regression-inference-transforming     1
math>>old-ap-calculus-ab>>ab-derivatives-advanced                            1
math>>4th-grade-foundations-engageny>>4th-m5-engage-ny-foundations           1
math>>algebra-basics>>alg-basics-expressions-with-exponents                  1
Name: hierarchy, Length: 569, dtype: int64

In [22]:
test_labels.value_counts()

science>>health-and-medicine>>human-anatomy-and-physiology                                    24
science>>health-and-medicine>>circulatory-system-diseases                                     22
science>>health-and-medicine>>circulatory-system                                              17
science>>health-and-medicine>>infectious-diseases                                             11
science>>biology>>crash-course-bio-ecology                                                    11
                                                                                              ..
math>>cc-seventh-grade-math>>cc-7th-fractions-decimals                                         1
math>>cc-sixth-grade-math>>cc-6th-equations-and-inequalities                                   1
science>>high-school-biology>>hs-evolution                                                     1
math>>in-in-class-5th-math-cbse>>x91a8f6d2871c8046:multi-digit-multiplication-and-division     1
math>>algebra-basics>>alg-basi

In [23]:

question_answer = train_features.values
categories = train_labels.values

In [24]:
question_answer

array([" In the last couple of videos we saw that we can describe a curves by a position vector-valued function. And in very general terms, it would be the x position as a function of time times the unit vector in the horizontal direction. Plus the y position as a function of time times the unit victor in the vertical direction. And this will essentially describe this-- though, if you can imagine a particle and let's say the parameter t represents time. It'll describe where the particle is at any given time. And if we wanted a particular curve we can say, well, this only applies for some curve-- we're dealing, it's r of t. And it's only applicable between t being greater than a and less than b. And you know, that would describe some curve in two dimensions. Just me just draw it here. This is all a review of really, the last two videos. So this curve, it might look something like that where this is where t is equal to a. That's where t is equal to b. And so r of a will be this vector ri

In [25]:
categories

array(['math>>multivariable-calculus>>multivariable-derivatives',
       'science>>ap-biology>>natural-selection',
       'math>>pre-algebra>>pre-algebra-equations-expressions', ...,
       'math>>engageny-geo>>geo-5',
       'science>>health-and-medicine>>human-anatomy-and-physiology',
       'math>>multivariable-calculus>>greens-theorem-and-stokes-theorem'],
      dtype=object)

In [26]:
# val_features = test_features.values
# val_labels = test_labels.values

In [27]:
# list(set(label_emb_data))[0]
# # list(set(train_data['board_syllabus'].values))[0]
# label_emb_data[0]

In [28]:

import numpy as np
!pip install inflection

from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.models import HoverTool
from bokeh import colors
import inflection

from nltk.stem import PorterStemmer 
ps = PorterStemmer()
from gzip import open as gopen
from pandas.core.common import flatten
import gensim.models.poincare as poincare
def get_cleaned_taxonomy(taxonomy):
  cleaned_taxonomy = []
  for value in taxonomy:
      value = ' '.join(value.lower().split(">>"))
      # taxonomy_words = [inflection.singularize(val)  for token in value for val in token.split(" ") if val.isalpha()]
      cleaned_taxonomy.append( value )
  return cleaned_taxonomy

In [29]:

# course_taxonomy

label_emb_data = get_cleaned_taxonomy(categories)
label_val = get_cleaned_taxonomy(val_labels)

In [30]:
label_emb_data[0]

'math multivariable-calculus multivariable-derivatives'

In [31]:
label_emb_data[2]

'math pre-algebra pre-algebra-equations-expressions'

In [32]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
wnl = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
label_input_ids = []
label_attention_masks = []
for sent in label_emb_data:

    label_encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    label_input_ids.append(label_encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    label_attention_masks.append(label_encoded_dict['attention_mask'])
label_input_ids = torch.cat(label_input_ids, dim=0)
label_attention_masks = torch.cat(label_attention_masks, dim=0)

In [34]:
label_input_ids_val = []
label_attention_masks_val = []
for sent in label_val:

    label_encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    label_input_ids_val.append(label_encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    label_attention_masks_val.append(label_encoded_dict['attention_mask'])
label_input_ids_val = torch.cat(label_input_ids_val, dim=0)
label_attention_masks_val = torch.cat(label_attention_masks_val, dim=0)

In [35]:
test_data

,video_transcripts,hierarchy
0,- What I hope to do in this video is get fam...,math>>math1>>x89d82521517266d4:functions
1,In the last video we were able to set up this...,math>>old-ap-calculus-ab>>ab-applications-defi...
2,- In previous videos we talk about GDP as th...,economics-finance-domain>>ap-macroeconomics>>e...
3,- So what we're gonna do in this video is se...,math>>old-integral-calculus>>definite-integral...
4,- So I've said that if you have a vector fie...,math>>multivariable-calculus>>multivariable-de...
...,...,...
1042,"In the last video, we saw that if a vector fi...",math>>multivariable-calculus>>integrating-mult...
1043,So I have the function g of x is equal to 9 t...,math>>algebra-home>>alg-sequences
1044,Let's see if we can get a little bit more pra...,science>>in-in-class11th-physics>>in-in-system...
1045,Find the probability of rolling even numbers ...,math>>precalculus>>x9e81a4f98389efdf:prob-comb


In [36]:
# np.array(poincare_embedding).shape

In [37]:
# taxonomy_vectors = []
# for label_input_id,label_att_mask in zip(label_input_ids,label_attention_masks):
#     label_input_id = label_input_id.to(device)
#     label_att_mask = label_att_mask.to(device)
#     with torch.no_grad():
#       outputs = model_label(label_input_id.reshape(1,-1),label_att_mask.reshape(1,-1))
#     taxonomy_vectors.append(outputs.cpu().numpy())
# taxonomy_vectors = np.vstack(taxonomy_vectors)
# taxonomy_vectors.shape


In [38]:
num_classes = len(list(set(categories)))
num_classes

569

In [39]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split


In [40]:
# !pip install transformers==2.8.0
import transformers
print(transformers.__version__)

2.8.0


In [41]:
set(question_answer).intersection(set(test_features))

set()

In [42]:

import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint

from tqdm import tqdm
import geoopt
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import torch.nn.functional as F

import time
import argparse
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
# Neural Classifierwork
class MulticlassClassifier(nn.Module):
    def __init__(self,bert_model_path):
        super(MulticlassClassifier,self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_path,output_hidden_states=False,output_attentions=False)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(768, 384)
        self.fc2 = nn.Linear(384, 768)

    def forward(self,tokens,masks):
        _, pooled_output = self.bert(tokens, attention_mask=masks)
        x = self.fc1(pooled_output)
        x = self.fc2(x)
        return x
class MyHingeLoss(torch.nn.Module):
    def __init__(self, margin):
        super(MyHingeLoss, self).__init__()
        self.margin = margin
    # def forward_val(self, output, target):
    #     cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    #     loss = 0
    #     num_compare = 4
    #     count = 0
    #     for i in range(len(output)):
    #         v_image = output[i]
    #         t_label = target[i]
    #         for j in range(num_compare):
    #             if j != i:
    #                 count += 1
    #                 t_j = target[j]
    #                 loss += torch.relu( self.margin - cos(t_label, v_image) + cos(t_j, v_image) )
    #     return loss / count

    def forward(self, output, target):
        loss=0
        for i in range(len(output)):
            v_image = output[i]
            t_label = target[i]
            j = randint(0, len(output)-1)
            while j == i:
                j = randint(0, len(output)-1)
            t_j = target[j]
            loss+= torch.relu( self.margin - cos(t_label, v_image) + cos(t_j, v_image) )
        return loss / len(output)


In [43]:
from transformers import BertModel, AdamW, BertConfig
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


In [44]:
from transformers import BertModel, AdamW, BertConfig

# Loads BertModel, the pretrained BERT model with a single 
model = MulticlassClassifier('bert-base-uncased')
model.load_state_dict(torch.load('model_euclidean_dual_bert_1_khan_acad/model_weights'))
model_label = MulticlassClassifier('bert-base-uncased')
# Tell pytorch to run this model on the GPU.
model_label.load_state_dict(torch.load('model_euclidean_dual_bert_2_khan_acad/model_weights'))

model.cuda()
model_label.cuda()

MulticlassClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [45]:
test_features = test_features.values
labels = test_labels.values

In [46]:
test_features

array([' -  What I hope to do in this video is get familiar with the notion of an interval, and also think about ways that we can show an interval, or interval notation. Right over here I have a number line. Let\'s say I wanted to talk about the interval on the number line that goes from negative three to two. So I care about this-- Let me use a different color. Let\'s say I care about this interval right over here. I care about all the numbers from negative three to two. So in order to be more precise, I have to be clear. Am I including negative three and two, or am I not including negative three and two, or maybe I\'m just including one of them. So if I\'m including negative three and two, then I would fill them in. So this right over here, I\'m filling negative three and two in, which means that negative three and two are part of this interval. And when you include the endpoints, this is called a closed interval. Closed interval. And I just showed you how I can depict it on a number

In [47]:
labels

array(['math>>math1>>x89d82521517266d4:functions',
       'math>>old-ap-calculus-ab>>ab-applications-definite-integrals',
       'economics-finance-domain>>ap-macroeconomics>>economic-iondicators-and-the-business-cycle',
       ...,
       'science>>in-in-class11th-physics>>in-in-system-of-particles-and-rotational-motion',
       'math>>precalculus>>x9e81a4f98389efdf:prob-comb',
       'science>>biology>>dna-as-the-genetic-material'], dtype=object)

In [48]:
len(list(set(labels)))

416

In [49]:
test_labels = list(set(labels))
test_set_labels = get_cleaned_taxonomy(test_labels)
test_set_labels[0]

'math linear-algebra alternate-bases'

In [50]:
test_labels[0]

'math>>linear-algebra>>alternate-bases'

In [51]:
label_input_ids = []
label_attention_masks = []
for sent in test_set_labels:

    label_encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    label_input_ids.append(label_encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    label_attention_masks.append(label_encoded_dict['attention_mask'])
label_input_ids = torch.cat(label_input_ids, dim=0)
label_attention_masks = torch.cat(label_attention_masks, dim=0)

In [52]:
taxonomy_vectors = []
for label_input_id,label_att_mask in zip(label_input_ids,label_attention_masks):
    label_input_id = label_input_id.to(device)
    label_att_mask = label_att_mask.to(device)
    with torch.no_grad():
      outputs = model_label(label_input_id.reshape(1,-1),label_att_mask.reshape(1,-1))
    taxonomy_vectors.append(outputs.cpu().numpy())
taxonomy_vectors = np.vstack(taxonomy_vectors)
taxonomy_vectors.shape


(416, 768)

In [53]:
# model = MulticlassClassifier('bert-base-uncased')
# # model.load_state_dict(torch.load('model_euclidean_cos/model_weights'))
# model.cuda()

In [54]:
test_input_ids = []
test_attention_masks = []
for sent in test_features:

    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
# labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  
test_poincare_tensor = torch.tensor(taxonomy_vectors,dtype=torch.float)

# Create the DataLoader.
# prediction_data = TensorDataset(test_input_ids, test_attention_masks, test_poincare_tensor)
# prediction_sampler = SequentialSampler(prediction_data)
# prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [55]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
test_poincare_tensor = torch.tensor(taxonomy_vectors,dtype=torch.float)


In [56]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
def dist_without_grad( u, v):
  sqdist = torch.sum((u - v) ** 2, dim=-1)
  squnorm = torch.sum(u ** 2, dim=-1)
  sqvnorm = torch.sum(v ** 2, dim=-1)
  x = 1 + 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm)) + 1e-7
  z = torch.sqrt(x ** 2 - 1)
  return torch.log(x + z)

In [57]:
test_labels = np.array(test_labels)

In [58]:
label_set = np.array(list(set(categories)))

In [59]:
test_poincare_tensor.shape

torch.Size([416, 768])

In [60]:
len(label_set)

569

In [93]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))

# Put model in evaluation mode
model.eval()
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

input_ids = test_input_ids.to('cuda')
attention_masks = test_attention_masks.to('cuda')
test_poincare_tensor = test_poincare_tensor.to('cuda')
# Tracking variables 
predictions , true_labels = [], []
for input_id,attention_mask in zip(input_ids, attention_masks):
  with torch.no_grad():
    outputs = model(input_id.reshape(1,-1),attention_mask.reshape(1,-1))
    
  distances = cos(outputs,test_poincare_tensor) #torch.topk(cos(outputs,test_poincare_tensor),20,largest=True)
  distances,indices = torch.topk(distances,10,largest=True)
  predictions.append(test_labels[indices.cpu().numpy()])
print(len(predictions))
  # max_distance =100000000000000
  # label=None
  # for index,test_poincare in enumerate(test_poincare_tensor):

  #   distance = distanceTo(test_poincare, outputs)
  #   if distance < max_distance:
  #     max_distance = distance
  #     label = index
  # predictions.append(labels[label])
    
# Predict 
# for batch in prediction_dataloader:
#   # Add batch to GPU
#   batch = tuple(t.to(device) for t in batch)
  
#   # Unpack the inputs from our dataloader
#   b_input_ids, b_input_mask, b_labels = batch
  
#   # Telling the model not to compute or store gradients, saving memory and 
#   # speeding up prediction
#   with torch.no_grad():
#       # Forward pass, calculate logit predictions
#       outputs = model(b_input_ids,b_input_mask)

#   logits = outputs
#   for logit in logits:
#     max_similarity = 0


#   # Move logits and labels to CPU
#   logits = logits.detach().cpu().numpy()
#   label_ids = b_labels.to('cpu').numpy()
  
#   # Store predictions and true labels
#   predictions.append(logits)
#   true_labels.append(label_ids)

print('    DONE.')
# predictions

Predicting labels for 1,047 test sentences...
1047
    DONE.


In [94]:
labels

array([238, 254,   2, ..., 386, 283, 324])

In [95]:
!pip install tensorflow==1.13.1

In [96]:
labels=test_data['hierarchy'].values

In [97]:
from sklearn .preprocessing import LabelEncoder
LE= LabelEncoder()
labels = LE.fit_transform(labels)
labels

array([238, 254,   2, ..., 386, 283, 324])

In [98]:
LE.inverse_transform([325])

array(['science>>biology>>ecology'], dtype=object)

In [99]:
labels

array([238, 254,   2, ..., 386, 283, 324])

In [100]:
final_predictions = []
for prediction in predictions:
  final_predictions.append(LE.transform(prediction))


In [101]:
final_predictions[-3]

array([181, 179,  48, 183, 178, 177, 176, 185, 180, 148])

In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 20
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, k=20)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, k=20)

tmp_rank = tf.nn.top_k(y_pred, 20)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",s|ess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 20) (1047,)
update_recall:  0.5635148042024832
recall 0.5635148042024832
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 590.0, 457.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[276, 201, 197, ...,  25,  24,  23],
       [267, 263, 262, ...,  94,  93,  90],
       [ 20,  19,  18, ...,   2,   1,   0],
       ...,
       [298, 241, 207, ...,  71,  48,  30],
       [283, 239, 232, ...,  34,  31,  28],
       [373, 372, 371, ..., 301, 300, 299]]), indices=array([[ 5, 11, 19, ..., 10,  3,  6],
       [12,  7,  1, ...,  8,  6, 14],
       [ 1, 13, 18, ...,  8, 14, 12],
       ...,
       [15, 19, 11, ..., 18,  2, 17],
       [ 2, 10, 18, ...,  1,  6, 19],
       [19,  4,  5, ..., 13,  2,  3]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 15)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 15)

tmp_rank = tf.nn.top_k(y_pred, 15)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 15) (1047,)
precision 0.03260108245781598
update_recall:  0.4890162368672397
recall 0.4890162368672397
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 512.0, 535.0, 512.0, 15193.0]
TMP_RANK:  TopKV2(values=array([[276, 201, 196, ...,  25,  24,  23],
       [267, 263, 262, ...,  94,  93,  90],
       [ 20,  19,  17, ...,   2,   1,   0],
       ...,
       [207, 185, 183, ..., 148, 133,  48],
       [283, 239, 222, ...,  35,  34,  31],
       [372, 371, 370, ..., 301, 300, 299]]), indices=array([[ 5, 11, 14, ..., 10,  3,  6],
       [12,  7,  1, ...,  8,  6, 14],
       [ 1, 13,  2, ...,  8, 14, 12],
       ...,
       [11,  7,  3, ...,  9, 14,  2],
       [ 2, 10,  9, ...,  0,  1,  6],
       [ 4,  5, 11, ..., 13,  2,  3]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 10)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 10)

tmp_rank = tf.nn.top_k(y_pred, 10)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1047, 10) (1047,)
precision 0.03648519579751672
update_recall:  0.3648519579751671
recall 0.3648519579751671
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 382.0, 665.0, 382.0, 10088.0]
TMP_RANK:  [[ 28  26  27 ... 188 115  54]
 [217 262 256 ... 263  94  95]
 [  8  20  17 ...  15   2   5]
 ...
 [181 179  48 ... 185 180 148]
 [ 35  34 283 ... 145 149 222]
 [327 304 300 ... 323 303 343]]


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 5)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 5)

tmp_rank = tf.nn.top_k(y_pred, 5)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 5) (1047,)
precision 0.047182425978987586
update_recall:  0.23591212989493793
recall 0.23591212989493793
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 247.0, 800.0, 247.0, 4988.0]
TMP_RANK:  TopKV2(values=array([[193, 114, 113, 112, 110],
       [252, 251, 250, 248, 159],
       [ 20,  12,   6,   4,   3],
       ...,
       [185, 183, 182, 148,  30],
       [199, 188, 145, 141,  52],
       [327, 326, 325, 304, 299]]), indices=array([[3, 4, 0, 2, 1],
       [4, 2, 3, 1, 0],
       [4, 2, 3, 1, 0],
       ...,
       [4, 0, 2, 1, 3],
       [2, 1, 0, 4, 3],
       [1, 2, 0, 3, 4]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 1)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 1)

tmp_rank = tf.nn.top_k(y_pred, 1)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(4784, 1) (4784,)
precision 0.4280936454849498
update_recall:  0.4280936454849498
recall 0.4280936454849498
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2048.0, 2736.0, 2048.0, 2736.0]
TMP_RANK:  TopKV2(values=array([[ 66],
       [176],
       [116],
       ...,
       [ 49],
       [  8],
       [152]]), indices=array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32))


In [ ]:
y_true = np.array(labels)
final_predictions = np.array(final_predictions).squeeze()
final_predictions.shape
len(final_predictions[final_predictions==y_true])/len(labels)

0.43415551839464883

In [ ]:
categories

In [ ]:
!cp /content/model_euclidean_cos.zip "/content/drive/My Drive/research_lo_content_taxonomy_classification"

print('Positive samples: %d of %d (%.2f%%)' % (final_data.label.sum(), len(final_data.label), (final_data.label.sum() / len(final_data.label) * 100.0)))

In [ ]:
pred =  np.argmax(predictions[0],axis=1).flatten()
pred

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  # pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(np.array(labels[i]), np.array(predictions[i])   )             
  matthews_set.append(matthews)

In [ ]:
ax = sns.barplot(x=list(range(len(matthews_set))), y=matthews_set, ci=None)

plt.title('MCC Score per Batch')
plt.ylabel('MCC Score (-1 to +1)')
plt.xlabel('Batch #')

plt.show()

In [ ]:
flat_predictions = np.array(predictions)
from sklearn.metrics import matthews_corrcoef

# flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.array(labels)

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

Total MCC: 0.000


In [ ]:
list_bool = (flat_true_labels==flat_predictions)
print(list_bool)
print(len([i for i, val in enumerate(list_bool) if val]))
len(flat_predictions)

[False False False ... False False False]
68


2664

In [ ]:
print('Total MCC: %.3f' % mcc)

Total MCC: 0.023


In [ ]:
len(flat_predictions[flat_predictions==flat_true_labels])/len(flat_predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


0.0

In [ ]:
flat_predictions[:40]

array(['social science>>civics : social and political life - ii>>how the state government works',
       'physical science>>physical science (chemistry)>>synthetic fibres and plastics>>plastics',
       'science>>physical and chemical changes',
       'social science>>civics : social and political life - ii>>how the state government works',
       'computer science[c++]>>arrays',
       'computer science[c++]>>standard library functions',
       'computer science[c++]>>inheritance: extending classes',
       'computer science[c++]>>inheritance: extending classes',
       'science>>reproduction in animals',
       'social science>>history : our pasts - iii>>weavers, iron smelter & factory owners',
       'social science>>civics : social and political life-i>>key elements of a democratic government',
       'science>>materials : metals and non-metals',
       'physics>>physics : part - i>>physical world',
       'computer science[c++]>>inheritance: extending classes',
       'science>>so

In [ ]:
flat_true_labels[:40]

array(['social science>>civics : social and political life - ii>>how the state government works',
       'science>>electricity and circuits', 'science>>changes around us',
       'social science>>civics : social and political life - ii>>understanding advertising',
       'computer science[c++]>>programming methodology',
       'computer science[c++]>>structured query language',
       'computer science[c++]>>object oriented programming',
       'computer science[c++]>>general oop concepts',
       'science>>how do organisms reproduce?',
       'social science>>history : our pasts - iii>>ruling the countryside',
       'social science>>civics : social and political life>>the indian constitution',
       'physical science>>physical science (chemistry)>>metals and non-metals>>metals',
       'physics>>physics : part - i>>motion in straight line',
       'computer science[c++]>>structured query language',
       'physics>>physics : part - i>>laws of motion',
       'social science>>civics 